In [129]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from ipywidgets import interact

# ODE-based Simulation Model Checkpoint
## 1. Simple SEIR model
The simulation function can be written as 

$$\frac{d S}{d t} =\mu(N-S)-\beta \frac{S I}{N}$$

$$\frac{d E}{d t} =\beta \frac{S I}{N}-(\mu+\sigma) E$$

$$\frac{d I}{d t} =\sigma E-(\mu+\gamma) I$$

$$\frac{d R}{d t} =\gamma I-\mu R$$
where $S, E, I, R$ stands for the *Susceptible*, *Exposed*, *Infected*, *Recovered* people respectively. 

- $N = S+E+I+R$ is the total number of population, 
- $\beta$ controls how often a susceptible-infected contact results in a new exposure, 
- $\gamma$ is	the rate an infected recovers and moves into the resistant phase, 
- $\sigma$ is the rate an exposed person becomes infective, 
- $\mu$ is the natural mortality rate (unrelated to disease).

In [130]:
def seir_base_simulation(N = 1000, 
                         I0 = 5, 
                         R0 = 0, 
                         E0 = 5, 
                         beta = 0.8, 
                         gamma = 0.1, 
                         sigma = 0.3, 
                         mu = 0.0001, 
                         t = 100):

    # Total population, N.
    #N = N
    # Initial number of infected and recovered individuals, I0 and R0.
    #I0, R0 = 1, 0
    # Everyone else, S0, is susceptible to infection initially.
    S0 = N - I0 - R0 - E0
    # Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
    # A grid of time points (in days)
    t = np.linspace(0, t, t)

    # The SIR model differential equations.
    def deriv(y, t, N, beta, gamma, sigma, mu):
        S, E, I, R = y
        beta = beta # * np.exp(-t/50)
        dSdt = mu * (N - S) - beta * S * I / N
        dEdt = beta * S * I / N - (mu + sigma) * E
        dIdt = sigma * E - (mu + gamma) * I
        dRdt = gamma * I - mu * R
        return dSdt, dEdt, dIdt, dRdt

    # Initial conditions vector
    y0 = S0, E0, I0, R0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma, sigma, mu))
    S, E, I, R = ret.T

    # Plot the data on three separate curves for S(t), I(t) and R(t)
    fig = plt.figure(facecolor='w', figsize = [9, 6])
    ax = fig.add_subplot(111,  axisbelow=True)
    ax.plot(t, S/N, 'b', alpha=0.6, lw=2.5, label='Susceptible')
    ax.plot(t, E/N, 'purple', alpha=0.6, lw=2.5, label='Exposed')
    ax.plot(t, I/N, 'r', alpha=0.6, lw=2.5, label='Infected')
    ax.plot(t, R/N, 'g', alpha=0.6, lw=2.5, label='Recovered')
    ax.set_xlabel('Time /days')
    ax.set_ylabel('Ratio of people')
    ax.set_ylim(0,1.1)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major',  c='grey',lw=1, ls=':')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()

# Simulation with interaction.
By changing different values on different parameters, the shape of curve will be different.

In [131]:
interact (seir_base_simulation
    , N=(1, 1000, 1)
    , I0=(1, 50, 1)
    , E0=(0, 100,1)
    , beta=(0, 1, 0.05)
    , gamma = (0, 0.5, 0.01)
    , sigma = (0, 1, 0.05)
    , mu = (0, 0.01, 0.001)
    , t = (0, 300, 10)
    );


interactive(children=(IntSlider(value=1000, description='N', max=1000, min=1), IntSlider(value=5, description=…

# More Complex Model: Considering Quarantine
Apart from the existing modules in SIRS model, we also considers the effect of self-quaratine, the infected/exposed people will stay at home to avoid interacting with others.
Specifically, we add to extra parameter $Q_E$, $Q_I$ which stand for the number of exposed/infected population being quarantined respectively. Then, the ODE formula can be expressed as 
$$\frac{d S}{d t} =\mu(N-S)-\beta \frac{S I}{N}$$

$$\frac{d E}{d t} =\beta \frac{S I}{N}-(\mu+\sigma+q) E$$

$$\frac{d I}{d t} =\sigma E-(\mu+\gamma+q) I$$

$$\frac{d R}{d t} =\gamma (I+Q_I)-\mu R$$

$$\frac{d Q_E}{d t} =- \sigma  Q_E + q  E$$

$$\frac{d Q_I}{d t} =- \gamma  Q_I + \sigma Q_E + q I$$

- $q$ is the proportion of population being self-quarantined.

In [132]:
def seir_add_simulation(N = 1000, 
                         I0 = 1, 
                         R0 = 0, 
                         E0 = 0, 
                         beta = 0.8, 
                         gamma = 0.1, 
                         sigma = 0.3, 
                         mu = 0.0001, 
                         q = 0.05,
                         t = 100):

    # Total population, N.
    #N = N
    # Initial number of infected and recovered individuals, I0 and R0.
    #I0, R0 = 1, 0
    # Everyone else, S0, is susceptible to infection initially.
    QE0, QI0 = 0, 0
    S0 = N - I0 - R0 - E0 - QE0 - QI0
    
    # Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
    # A grid of time points (in days)
    t = np.linspace(0, t, t)

    # The SIR model differential equations.
    def deriv(y, t, N, beta, gamma, sigma, mu, q):
        S, E, I, QE, QI, R = y
        dSdt = mu * (N - S) -beta * S * I / N
        dEdt = beta * S * I / N - (mu + sigma + q) * E
        dIdt = sigma * E - (mu + gamma + q) * I
        dRdt = gamma * I - mu * R + gamma * QI
        dQEdt = - sigma * QE + q * E
        dQIdt = - gamma * QI + sigma * QE + q * I
        return dSdt, dEdt, dIdt, dQEdt, dQIdt, dRdt

    # Initial conditions vector
    y0 = S0, E0, I0, QE0, QI0, R0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma, sigma, mu, q))
    S, E, I, QE, QI, R = ret.T

    # Plot the data on three separate curves for S(t), I(t) and R(t)
    fig = plt.figure(facecolor='w', figsize = [9, 6])
    ax = fig.add_subplot(111,  axisbelow=True)
    ax.plot(t, S/N, 'b', alpha=0.6, lw=2.5, label='Susceptible')
    ax.plot(t, (E+QE)/N, 'purple', alpha=0.6, lw=2.5, label='Exposed')
    ax.plot(t, (I+QI)/N, 'r', alpha=0.6, lw=2.5, label='Infected')
    ax.plot(t, R/N, 'g', alpha=0.6, lw=2.5, label='Recovered')
    #ax.plot(t, QE/N, 'hotpink', alpha=0.6, lw=2.5, label='Recovered')
    #ax.plot(t, QI/N, 'orange', alpha=0.6, lw=2.5, label='Recovered')
    ax.set_xlabel('Time /days')
    ax.set_ylabel('Ratio of people')
    ax.set_ylim(0,1.1)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major',  c='grey',lw=1, ls=':')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()

Simulation with interaction.
By changing different values on different parameters, the shape of curve will be different.
Compare our model with different $q$, it is clear that quarantine will reduce the total proportion of infected population.

In [134]:
interact (seir_add_simulation
    , N=(1, 1000, 1)
    , I0=(1, 50, 1)
    , E0=(0, 100,1)
    , beta=(0, 1, 0.05)
    , gamma = (0, 0.5, 0.01)
    , sigma = (0, 1, 0.05)
    , mu = (0, 0.01, 0.001)
    , q = (0, 1, 0.05)
    , t = (0, 300, 10)
    );


interactive(children=(IntSlider(value=1000, description='N', max=1000, min=1), IntSlider(value=1, description=…